In [40]:
#import modules
import json
import pandas as pd
from binance.client import Client
from dotenv import load_dotenv
from envs import env
import time
import re
import os
import datetime
from datetime import timedelta
import statistics
import numpy as np
from scipy.stats import kurtosis, skew

In [41]:
#import classes from ./ folder
import postgresdbAccess

In [42]:
#read fibonacci retracements  from json
with open('fibLvl.json') as file:
    fibLvl = json.load(file)

In [43]:
def backtest():
    #initiate empty open positions
    openPositions = {}
    #initiate postgresdb class
    postgres = postgresdbAccess.postgresAccess()
    #get unique simbol list
    sql = ("SELECT distinct symbol FROM " + dbTable + ";")
    uniqueSymbol = pd.DataFrame(postgres.sqlQuery(sql))
    uniqueSymbol = uniqueSymbol[0]
    #initiate empty percent bar
    percentBar = 0
    #loop over every symbol in symbol list
    for symbol in uniqueSymbol:
        #print progress
        print(percentBar, " / ", len(uniqueSymbol))
        percentBar = percentBar + 1
        #skip if symbol is not to be considered
        #null askprice is sorted out with crawler already
        if not (symbol.endswith(baseCurrency) and
               len(symbol) < 11):
            continue
        #check if open position was left over from last symbol and simulate close
        if openPositions:
            #prepare sql for close
            openPositions['resultPercent'] = ((row[4] - openPositions['askPrice']) / openPositions['askPrice']) * 100 - brokerFees * 2.55
            openPositions['stopId'] = row[0]
            closeSql = ("UPDATE " + dbTable + " SET" +
            "', resultpercent = '" + str(openPositions['resultPercent']) +
            "', stopid = '" + str(openPositions['stopId']) +
            "' WHERE id = '" + str(int(openPositions['id'])) +
            "';")
            postgres.sqlUpdate(closeSql)
            openPositions = {}
        #query for every line for symbol
        sql = ("SELECT id, time, symbol, askprice, " +
               "bidprice, highprice, quotevolume, " +
               "pricechangepercent, weightedavgprice, openprice" +
            " FROM " + dbTable + " WHERE" +
            " symbol like '" + symbol + "' order by id;")
        print(sql)
        bigData = pd.DataFrame(postgres.sqlQuery(sql))
        bigData[0] = pd.to_numeric(bigData[0], errors='coerce', downcast='float')
        bigData[1] = pd.to_datetime(bigData[1])
        bigData[3] = pd.to_numeric(bigData[3], errors='coerce', downcast='float')
        bigData[4] = pd.to_numeric(bigData[4], errors='coerce', downcast='float')
        bigData[6] = pd.to_numeric(bigData[6], errors='coerce', downcast='float')
        bigData[7] = pd.to_numeric(bigData[7], errors='coerce', downcast='float')
        bigData[8] = pd.to_numeric(bigData[8], errors='coerce', downcast='float')
        bigData[9] = pd.to_numeric(bigData[9], errors='coerce', downcast='float')
        #get start of timedelta
        bigData[5] = bigData[1] - timedelta(hours=33)
        bigData[10] = bigData[1] - timedelta(hours=22)
        bigData[11] = bigData[1] - timedelta(hours=11)
        #loop over every row
        for index, row in bigData.iterrows():
            before_start_date = bigData[1] <= row[5]
            if len(bigData.loc[before_start_date]) > 0:
                #get data for consideration
                after_start_date = bigData[1] >= row[5]
                before_end_date = bigData[1] <= row[1]
                between_two_dates = after_start_date & before_end_date
                fibDates = bigData.loc[between_two_dates]
                diff = fibDates[3].max() - fibDates[3].min()
                # calculate fibRetracements
                fibRetracement = pd.DataFrame(fibLvl)
                maxAsk = fibDates[3].max()
                for lvl in fibRetracement:
                    fibRetracement[1] =  maxAsk - diff * fibRetracement[0]
                    fibRetracement[2] = fibRetracement[1] * 0.9995
                    fibRetracement[3] = fibRetracement[1] * 1.0005
                #calculate corvalue
                corValue = fibDates[0].corr(fibDates[3])
                before_start_date1 = bigData[1] <= row[5]
                bigData[10] = bigData[1] - timedelta(hours=15)
                after_start_date1 = bigData[1] >= row[10]
                before_end_date1 = bigData[1] <= row[1]
                between_two_dates1 = after_start_date1 & before_end_date1
                fibDates1 = bigData.loc[between_two_dates]
                corValue1 = fibDates1[0].corr(fibDates1[3])
                
                before_start_date2 = bigData[1] <= row[5]
                bigData[11] = bigData[1] - timedelta(hours=5)
                after_start_date2 = bigData[1] >= row[11]
                before_end_date2 = bigData[1] <= row[1]
                between_two_dates2 = after_start_date2 & before_end_date2
                fibDates2 = bigData.loc[between_two_dates2]
                corValue2 = fibDates2[0].corr(fibDates2[3])
                
                #get statistical parameters
                statistics = {}
                statistics["stdev"] = statistics.stdev(fibDates[3])
                statistics["skewness"] = skewness(fibDates[3])
                statistics["kurtosis"] = kurtosis(fibDates[3])
                
                #if an open position exists, check if it can be closed
                if openPositions:
                    if (row[3] < openPositions['stopLoss'] or
                        row[3] > openPositions['takeProfit']):
                        #prepare sql for close
                        openPositions['resultPercent'] = ((row[4] - openPositions['askPrice']) / openPositions['askPrice']) * 100 - brokerFees * 2
                        openPositions['stopId'] = row[0]
                        closeSql = ("UPDATE " + dbTable + " SET" +
                        " startId = '" + str(int(openPositions['startId'])) +
                        "', fibLevel = '" + str(openPositions['fibLevel']) +
                        "', midId = '" + str(int(openPositions['id'])) +
                        "', corValue = '" + str(openPositions['corValue']) +
                        "', takeProfit = '" + str(openPositions['takeProfit']) +
                        "', stopLoss = '" + str(openPositions['stopLoss']) +  
                        "', resultpercent = '" + str(openPositions['resultPercent']) +
                        "', stopid = '" + str(openPositions['stopId']) +
                        "', stdev = '" + str(openPositions['stDev']) +
                        "', skewness = '" + str(openPositions['skewness']) +
                        "', kurtosis = '" + str(openPositions['kurtosis']) +
                        "' WHERE id = '" + str(int(openPositions['id'])) +
                        "';")
                        #update database
                        postgres.sqlUpdate(closeSql)
                        #clear open position
                        openPositions = {}
                else:
                    #loop over considered fibonacciretracements
                    for i in [7]:
                        #check if buy requirements are met
                        if (corValue >= 0 and
                            corValue1 >= 0 and
                            corValue2 >= 0 and
                            row[3] < fibRetracement[3][i] and
                            row[3] > fibRetracement[2][i]):
                            openPositions['startId'] = fibDates[0].min()
                            openPositions['id'] = row[0]
                            openPositions['bidPrice'] = row[4]
                            openPositions['fibLevel'] = i
                            openPositions['symbol'] = symbol
                            openPositions['askPrice'] = row[3]
                            openPositions['corValue'] = corValue
                            openPositions['stDev'] = statistics.stdev(statistics["stDev"])
                            openPositions['skewness'] = statistics.stdev(statistics["skewness"])
                            openPositions['kurtosis'] = statistics.stdev(statistics["kurtosis"])
                            openPositions['takeProfit'] = fibRetracement[3][i+4]
                            openPositions['stopLoss'] = fibRetracement[2][i-1]
    #close database connection
    postgres.databaseClose()

In [39]:
testTables = [{"table":"backtesting","currency":"BNB"},
             {"table":"backtesting","currency":"BTC"},
             {"table":"backtesting","currency":"ETH"},
             {"table":"backtesting","currency":"ADA"}]
for iteration in testTables:
    load_dotenv('../.env')
    try:
        liveVolume=env("liveVolume")
        apiSecret=env('apiSecret')
        apiKey=env('apiKey')
        dbTable=iteration["table"]
        baseCurrency=iteration["currency"]
        brokerFees=float(env('brokerFees'))
        liveTrading=env("liveTrading", 'False').lower() in ('true', '1', 't')
    except KeyError:
        print("No env variables set.")
        sys.exit(1)
    client = Client(apiKey, apiSecret, {'timeout':600})
    backtest()

0  /  752
1  /  752
2  /  752
3  /  752
4  /  752
5  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'ADXBNB' order by id;
6  /  752
7  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'ENJBNB' order by id;
8  /  752
9  /  752
10  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'CNDBNB' order by id;
11  /  752
12  /  752
13  /  752
14  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'RDNBNB' order by id;
15  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHER

116  /  752
117  /  752
118  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'CVCBNB' order by id;
119  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'TORNBNB' order by id;
120  /  752
121  /  752
122  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'XVSBNB' order by id;
123  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'FLMBNB' order by id;
124  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'MLNBNB' order by id;
125  /  752
SELECT id

225  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'MINABNB' order by id;
226  /  752
227  /  752
228  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'KP3RBNB' order by id;
229  /  752
230  /  752
231  /  752
232  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'NUBNB' order by id;
233  /  752
234  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'ANKRBNB' order by id;
235  /  752
236  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'MBLBN

351  /  752
352  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'CREAMBNB' order by id;
353  /  752
354  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'WINBNB' order by id;
355  /  752
356  /  752
357  /  752
358  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'WINGBNB' order by id;
359  /  752
360  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'ONTBNB' order by id;
361  /  752
362  /  752
363  /  752
364  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting

491  /  752
492  /  752
493  /  752
494  /  752
495  /  752
496  /  752
497  /  752
498  /  752
499  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'RCNBNB' order by id;
500  /  752
501  /  752
502  /  752
503  /  752
504  /  752
505  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'FETBNB' order by id;
506  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'TCTBNB' order by id;
507  /  752
508  /  752
509  /  752
510  /  752
511  /  752
512  /  752
513  /  752
514  /  752
515  /  752
516  /  752
517  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'P

...

107  /  752
108  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'DOCKETH' order by id;
109  /  752
110  /  752
111  /  752
112  /  752
113  /  752
114  /  752
115  /  752
116  /  752
117  /  752
118  /  752
119  /  752
120  /  752
121  /  752
122  /  752
123  /  752
124  /  752
125  /  752
126  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'EVXETH' order by id;
127  /  752
128  /  752
129  /  752
130  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'STMXETH' order by id;
131  /  752
132  /  752
133  /  752
134  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE 

285  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'NASETH' order by id;
286  /  752
287  /  752
288  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'APPCETH' order by id;
289  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'GRTETH' order by id;
290  /  752
291  /  752
292  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'EASYETH' order by id;
293  /  752
294  /  752
295  /  752
296  /  752
297  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol

446  /  752
447  /  752
448  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'GHSTETH' order by id;
449  /  752
450  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'WTCETH' order by id;
451  /  752
452  /  752
453  /  752
454  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'EOSETH' order by id;
455  /  752
456  /  752
457  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'ZRXETH' order by id;
458  /  752
459  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting W

589  /  752
590  /  752
591  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'FRONTETH' order by id;
592  /  752
593  /  752
594  /  752
595  /  752
596  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'DATAETH' order by id;
597  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'VENETH' order by id;
598  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'SNGLSETH' order by id;
599  /  752
600  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'CH

KeyboardInterrupt: 